In [1]:
import argparse
import numpy as np
from pprint import pprint
from tqdm import tqdm
from PIL import Image
import matplotlib.pyplot as plt
import torch.optim as optim
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import grad
import torchvision
from torchvision import models, datasets, transforms
# from utils import label_to_onehot, cross_entropy_for_onehot, pixelwise_euclidean_distance
from whitebox_utils import label_to_onehot, cross_entropy_for_onehot, pixelwise_euclidean_distance

In [2]:

device = "cpu"
if torch.cuda.is_available():
    device = "cuda"
print("Running on %s" % device)
print(torch.__version__)
print(torchvision.__version__)

Running on cuda
2.3.1+cu121
0.18.1+cu121


In [3]:
parser = argparse.ArgumentParser(description='Deep Leakage from Gradients.')
parser.add_argument('--index', type=int, default=25,  
                    help='the index for leaking images on GTSRB.')
parser.add_argument('--image', type=str, default="",  
                    help='the path to customized image.')

# 解析参数
args = argparse.Namespace(index=30)
print("Received arguments:", args)

Received arguments: Namespace(index=30)


In [4]:

tt = transforms.ToTensor()#转发为张量
tp = transforms.ToPILImage()#转换为图片
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor()
])
num_picture = 23
# 加载GTSRB数据集
# dst = datasets.CIFAR10(root="/mnt/workspace/magazine", download=True, transform=transform)

# img_index = args.index
# gt_data = dst[img_index+num_picture][0].to(device)  # 假设使用GPU，可按需调整
# gt_data = gt_data.view(1, *gt_data.size())  
# gt_label = torch.tensor([dst[img_index+num_picture][1]]).long().to(device)
# gt_label = gt_label.view(1, )
# gt_onehot_label = label_to_onehot(gt_label)
target_path = '/mnt/workspace/magazine/cat.png'



plt.imshow(tp(gt_data[0].cpu()))
plt.show()
print(gt_data.shape)

  1%|          | 950272/170498071 [00:43<2:10:43, 21616.50it/s]


KeyboardInterrupt: 

In [ ]:
print(gt_data.shape)
dummy_data = torch.randn(gt_data.size()).to(device).requires_grad_(True)#形状为（1，c，h，w）
dummy_label = torch.randn(gt_onehot_label.size()).to(device).requires_grad_(True)
plt.imshow(tp(dummy_data[0].cpu()))
plt.show()

In [ ]:
print(dummy_data.size())
# 总变差函数,输入dummy图像
def _tensor_size(t):
    return t.size()[1]*t.size()[2]*t.size()[3]
def total_variation(x):
    batch_size = x.size()[0]
    h_x = x.size()[2]
    w_x = x.size()[3]
    count_h = _tensor_size(x[:,:,1:,:])
    count_w = _tensor_size(x[:,:,:,1:])
    h_tv = torch.pow(x[:,:,1:,:]-x[:,:,:h_x-1,:], 2).sum()
    w_tv = torch.pow(x[:,:,:,1:]-x[:,:,:,:w_x-1], 2).sum()
    return (h_tv / count_h + w_tv / count_w) / batch_size

# def loss_function_1(X_cut, X0_cut,tv, lambda_param=0.4):

#     euclidean_dist = pixelwise_euclidean_distance(X_cut, X0_cut)
    
#     return euclidean_dist + lambda_param * tv

In [ ]:

from models.vision import weights_init
class VGG9(nn.Module):
    def __init__(self, num_classes, sp=4, in_channels=3, fig_size=32):####sp为切割点
        assert sp<=17 and sp>0
        super(VGG9, self).__init__()
        cfg = [32, 64, 128, 128, 256, 256, 512, 512]
        self.conv1 = nn.Conv2d(in_channels,  cfg[0], 3, 1, 1)
        self.conv2 = nn.Conv2d(cfg[0], cfg[1], 3, 1, 1)
        self.conv3 = nn.Conv2d(cfg[1], cfg[2], 3, 1, 1)
        self.conv4 = nn.Conv2d(cfg[2], cfg[3], 3, 1, 1)
        self.conv5 = nn.Conv2d(cfg[3], cfg[4], 3, 1, 1)
        self.conv6 = nn.Conv2d(cfg[4], cfg[5], 3, 1, 1)
        self.fc1   = nn.Linear(int((fig_size/8)**2)*cfg[5], cfg[6])
        self.fc2   = nn.Linear(cfg[6], cfg[7])
        self.fc3   = nn.Linear(cfg[7], num_classes)
        self.total_module_list = [self.conv1, self.conv2, nn.ReLU(), nn.MaxPool2d(2, 2), 
                                self.conv3, self.conv4, nn.ReLU(), nn.MaxPool2d(2, 2),
                                self.conv5, self.conv6, nn.ReLU(), nn.MaxPool2d(2, 2),
                               nn.Flatten(1), self.fc1, nn.ReLU(), self.fc2, nn.ReLU(), self.fc3]
        self.f = nn.Sequential(*self.total_module_list[0:sp])
        self.g = nn.Sequential(*self.total_module_list[sp:])

    def forward(self, x):
        x_1 = self.f(x)
        x_cut = x_1.detach().clone()
        x_out = self.g(x_1)
        return x_out, x_cut

net = VGG9(num_classes=10).to(device)#模型名字


In [ ]:
#optimizer.zero_grad()  # 清空梯度
torch.manual_seed(1234)

net.apply(weights_init)
criterion = cross_entropy_for_onehot

# compute original gradient 
original_cut = net.f(gt_data)
ture_cut= original_cut.detach().clone()
#print(original_cut)
# original_loss = criterion(pred, gt_onehot_label)
# dy_dx = torch.autograd.grad(original_loss, net.parameters(), retain_graph=True)

#original_dy_dx = list((_.detach().clone() for _ in dy_dx))


In [ ]:
print(dummy_data.size)
print(dummy_data.shape)
#print(original_loss)
#print(dy_dx)

In [ ]:
T = 20000
lr = 0.001
optimizer = torch.optim.Adam([dummy_data], lr=lr)
range_T = tqdm(range(1, T+1), desc="White-box attacking...")
epsilon=1e-6
l0 = 1e-3
l1 = 1
l2 = 1e-3
history = []


for iters in range_T:
    optimizer.zero_grad()  # 清空梯度
        # 使用网络对虚拟数据进行预测
    dummy_cut = net.f(dummy_data)
    tv = total_variation(dummy_data)
    diff_cut = torch.norm((dummy_cut[ture_cut.abs()>epsilon]- ture_cut[ture_cut.abs()>epsilon]), p=2)
    loss_1 = l0*torch.norm(dummy_data-torch.clamp(dummy_data, min=0, max=1))
    loss_2 = l2*torch.norm(dummy_cut[ture_cut.abs()<epsilon])
    dummy_loss = diff_cut +tv*l1+l0*loss_1 + l0*loss_2
        
        #dummy_dcut_dx = torch.autograd.grad(dummy_loss, net.parameters(), create_graph=True, retain_graph=True, allow_unused=True)
    dummy_loss.backward()
    optimizer.step()  # 进行dummy_data参数更新
    #dummy_loss, dummy_dcut_dx = closure()
   
    if iters % 1000 == 0: ##打印损失
        current_loss= dummy_loss#closure(dummy_data)
        #print(iters, "%.4f" % current_loss.item())
        history.append(tp(dummy_data[0].cpu()))
        print("dummy_loss value:",current_loss)
plt.figure(figsize=(12, 8))
for i in range(20):
    plt.subplot(2, 10, i + 1)
    plt.imshow(history[i])
    plt.title("虚拟图片展示（iter=%d)" % (i * 1000))
    plt.axis('off')

plt.show()


In [ ]:
tt = transforms.ToTensor()#转发为张量
tp = transforms.ToPILImage()#转换为图片
def image2prob(image_dummy):
    image_dummy_tensor = tt(image_dummy)
    # 计算三通道总和并对每个通道分别归一化
    total_sum = image_dummy_tensor.sum(dim=(1, 2))
    image_dummy_norm = image_dummy_tensor / total_sum.view(-1, 1, 1)
    dummy_prob = F.softmax(image_dummy_norm.view(-1), dim=0)
    return dummy_prob
temp_good = 1
kl_list = []
attack_image = dummy_data.detach().clone()
ture_prob = image2prob(tp(gt_data[0]))
for index, image_dummy in enumerate(history):
    dummy_prob = image2prob(image_dummy)
    
    kl_div = F.kl_div(ture_prob.log(), dummy_prob, reduction='batchmean')
    kl_div = torch.abs(kl_div)
    kl_list.append(kl_div.cpu())
    if kl_div < temp_good:
        temp_good = kl_div
        attack_image = image_dummy
        attack_image_index = index
print(attack_image_index)
print(kl_list)

plt.imshow(attack_image)
plt.show()

In [ ]:
import math
c = kl_list[0]
landa = 0.5
kl_model = []
for temp_2 in range(1, len(kl_list)):
    kl_temp = 1-kl_list[temp_2]/c
    
    kl_1 = abs(math.log(1+landa*kl_temp,2))
    print(kl_1)
    kl_model.append(kl_1)

In [ ]:
tt = transforms.ToTensor()#转发为张量
tp = transforms.ToPILImage()#转换为图片
def image2prob(image_dummy):
    image_dummy_tensor = tt(image_dummy)
    # 计算三通道总和并对每个通道分别归一化
    total_sum = image_dummy_tensor.sum(dim=(1, 2))
    image_dummy_norm = image_dummy_tensor / total_sum.view(-1, 1, 1)
    dummy_prob = F.softmax(image_dummy_norm.view(-1), dim=0)
    return dummy_prob
temp_good = 1
attack_image = dummy_data.detach().clone()
#print(type(gt_data))
image_dummy = history[9]
dummy_prob = image2prob(image_dummy)
ture_prob = image2prob(tp(gt_data[0]))
kl_div = F.kl_div(ture_prob.log(), dummy_prob, reduction='batchmean')
if kl_div < temp_good:
    temp_good = kl_div
    attack_image = image_dummy
plt.imshow(attack_image)
plt.show()

In [ ]:
print(kl_div)

In [ ]:
tt = transforms.ToTensor()#转发为张量
tp = transforms.ToPILImage()#转换为图片
text_data = torch.randn(gt_data.size()).to(device)#形状为（1，c，h，w）
text_prob = image2prob(tp(text_data[0]))
ture_prob = image2prob(tp(gt_data[0]))
kl_div = F.kl_div(ture_prob.log(), text_prob, reduction='batchmean')
print(kl_div)

In [ ]:
attack_list = [3.6107, 10.803, 30.751, 114.85, 967.5]
max_num = 2182.4
attack_list = [item/max_num for item in attack_list]
import matplotlib.pyplot as plt
bar_width = 0.35
r1 = range(len(attack_list))


plt.bar(r1, attack_list, width=bar_width, label='KL', color='blue')

plt.xlabel('Setpoint')
plt.ylabel('kl')
plt.title('kl vs Setpoint')
plt.legend()
plt.show()